In [21]:
import random
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
import folium
import math

In [33]:

boundary_gdf = gpd.read_file('new-orleans.geojson')
city_center = Point(-90.0715, 29.9511)  # New Orleans city center (Longitude, Latitude)
max_distance_miles = 7
# Get the bounds of the polygon
minx, miny, maxx, maxy = boundary_gdf.total_bounds

In [34]:
def haversine_distance(point1, point2):
    """
    Calculate the Haversine distance between two points.
    """
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y

    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 3958.8  # Radius of earth in miles
    return c * r

In [35]:
def generate_random_points_within_radius(polygons, num_points, min_distance):
    points = []
    attempts = 0
    max_attempts = num_points * 10  # To avoid an infinite loop

    while len(points) < num_points and attempts < max_attempts:
        # Generate random points within the bounding box
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        
        # Check if the point is within any of the polygons
        if any(polygon.contains(random_point) for polygon in polygons):
            # Check if the point is within the maximum distance from the city center
            if haversine_distance(city_center, random_point) <= max_distance_miles:
                # Check distance against all other points
                if all(random_point.distance(existing_point) >= min_distance for existing_point in points):
                    points.append(random_point)
                else:
                    attempts += 1  # Increase attempts if the point is too close to others
            else:
                attempts += 1  # Increase attempts if the point is outside the maximum distance
    return points

In [36]:
new_orleans_multipolygon = boundary_gdf.union_all()


In [37]:
# Check if the result is a MultiPolygon or a single Polygon
if isinstance(new_orleans_multipolygon, Polygon):
    # If it's a single Polygon, wrap it in a list to treat it like a MultiPolygon
    new_orleans_multipolygon = [new_orleans_multipolygon]
elif isinstance(new_orleans_multipolygon, MultiPolygon):
    # If it's a MultiPolygon, extract all the polygons
    new_orleans_multipolygon = list(new_orleans_multipolygon)


In [38]:
min_distance = 0.01
random_points = generate_random_points_within_radius(new_orleans_multipolygon, 300, min_distance)


In [39]:
for point in random_points:
    print(f"Latitude: {point.y}, Longitude: {point.x}")

Latitude: 29.956969553435577, Longitude: -90.11295734562061
Latitude: 30.011266383314485, Longitude: -90.07728191476632
Latitude: 29.91458695905421, Longitude: -90.01854855648119
Latitude: 29.975802650965527, Longitude: -90.04680638143522
Latitude: 29.96264442348628, Longitude: -90.09751262655034
Latitude: 30.01809077424076, Longitude: -89.98442624428031
Latitude: 29.971852894229123, Longitude: -90.03690439115188
Latitude: 30.028841295454917, Longitude: -90.04691587323578
Latitude: 30.01181191639416, Longitude: -90.11966468161673
Latitude: 30.01760026138856, Longitude: -90.05861767285111
Latitude: 29.999542353235103, Longitude: -89.97817294639168
Latitude: 29.947878525010246, Longitude: -90.10507838029203
Latitude: 29.95782043870246, Longitude: -90.08868048712522
Latitude: 29.94846736919194, Longitude: -90.02017057251763
Latitude: 30.000921829002785, Longitude: -90.03782173811194
Latitude: 29.980387029627142, Longitude: -89.98662946922387
Latitude: 30.02629403895424, Longitude: -90.026

In [40]:
map_center = [29.9511, -90.0715]
m = folium.Map(location=map_center, zoom_start=12)

In [41]:
# Add random points to the map
for point in random_points:
    folium.Marker(location=[point.y, point.x]).add_to(m)

# Save the map as an HTML file


In [42]:
m


In [32]:
len(random_points)

146

In [44]:
import pandas as pd

In [45]:
# Create a DataFrame from the random points
df = pd.DataFrame({
    'Latitude': [point.y for point in random_points],
    'Longitude': [point.x for point in random_points]
})

# Write the DataFrame to a CSV file
df.to_csv('nola.csv', index=False)

print("Points have been saved to nola.csv.")


Points have been saved to nola.csv.
